In [2]:
# Imports 

import pywhatkit as kit
import time
import qrcode
from PIL import Image
import firebase_admin
from firebase_admin import credentials, firestore
import hashlib
from datetime import datetime


In [3]:
# Initializations 
cred = credentials.Certificate('credentials2.json')
firebase_admin.initialize_app(cred)


In [19]:
db = firestore.client()
Registrations = db.collection('Registrations')
Tickets = db.collection('QR_Codes')
# IDCounter = db.get('counterDocument')
logo_link = 'logo2.png'
logo = Image.open(logo_link)
reg_write_template = {'Name': '', 'Date': datetime.now(), 'amount': 300, 'totalCount': 8, 'number': '9944524545', 'ticketID': '', 'generatedTicket': False, 'sentTicket': False, 'imageID': ''}
idcount_doc = db.collection('Logs_Variables').document('docCounter')


In [28]:
test_doc = db.collection('Registrations').document('holi_bulk_1').get().to_dict()
print(test_doc)
test_doc['sentTicket'] = True
# test_dict = test_doc.to_dict()


{'number': '9944524545', 'amount': 300, 'imageID': '', 'generatedTicket': False, 'sentTicket': False, 'totalCount': 8, 'Date': DatetimeWithNanoseconds(2024, 2, 1, 0, 59, 28, 759460, tzinfo=datetime.timezone.utc), 'ticketID': '', 'Name': 'Radha'}
{'number': '9944524545', 'amount': 300, 'imageID': '', 'generatedTicket': False, 'sentTicket': True, 'totalCount': 8, 'Date': DatetimeWithNanoseconds(2024, 2, 1, 0, 59, 28, 759460, tzinfo=datetime.timezone.utc), 'ticketID': '', 'Name': 'Radha'}


In [30]:
counter_doc = db.collection('Logs_Variables').document('docCounter')
counter_doc.update({'userCount': firestore.Increment(1)})

update_time {
  seconds: 1706795762
  nanos: 712480000
}
transform_results {
  integer_value: 2
}

In [31]:
tkt_template = {'totalCount' : 0, 'visitedCount' : 0, 'used':False, 'timestamp' : ''}

In [36]:
tkt_template['totalCount'] = db.collection('Registrations').document('holi_bulk_1').get().get('totalCount')
print(tkt_template)

{'totalCount': 8, 'visitedCount': 0, 'used': False, 'timestamp': ''}


In [16]:
# if idcount.exists:
            # Access the userID field from the document data
idcount = idcount_doc.get().get('userCount')
# user_id = idcount.get('userCount')

# Print the userID
print("User ID:", idcount)

User ID: 6


In [38]:
reg_doc = db.collection('Registrations').document('holi_bulk_1')
# Check the codition
if reg_doc.get().get('generatedTicket'):
    print('works')
else:
    print('this works too')

this works too


In [17]:
def hashing(msg):
    hashval = hashlib.sha1(msg.encode("UTF-8")).hexdigest()
    return hashval[:10]
def generate_ticket(data,template,ID):
    base = 'holi_event_'
    raw = base+data[1]
    ID = hashing(base+data[1])
    QRcode = qrcode.QRCode(error_correction=qrcode.constants.ERROR_CORRECT_H)
    QRcode.add_data(data)
    QRcode.make()

    QRcolor = 'Black'
    QRimg = QRcode.make_image(
        fill_color=QRcolor, back_color="white").convert('RGBA')

    pos = ((QRimg.size[0] - logo.size[0]) // 2,
        (QRimg.size[1] - logo.size[1]) // 2)

    # Paste the logo onto the QR code with transparency
    QRimg.paste(logo, pos, logo)

    # Save the QR code generated
    QRimg.save(f'Codes/{ID}.png')
        
    return ID
def send_message(ticketID,number,imageID):
    message = "Dear customer, thank you for choosing event managers for the holi event on 25th"
    kit.sendwhats_image("+91"+number, "Codes/imageID.png",message, 10, tab_close=True)
    time.sleep(4)
def write_columns(collection,columns,ID,data):
    to_set = {}
    doc_ref = collection.document(ID)
    for field in columns:
        if field in data:
            to_set[field] = data[field]
    doc_ref.set(to_set)
def get_all_data(): #Input the document to be read/processed
    collection_list = []
    fields = ['Name','number','generatedTicket','ticketID','totalCount','sentTicket']
    for doc in Registrations:
        if doc.exists:
            temp = []
            for field in fields:
                temp.append(doc.get(field))
            collection_list.append(temp)
    
    return collection_list
def ticket_not_generated(data):
    res = []
    for row in data:
        if row[2] == False:
            res.append(row)
        
    return res


In [22]:
for i in range(3):
    reg_docs = Registrations.get()
    ticket_docs = Tickets.get()
    # all = get_all_data()
    no_tickets = ticket_not_generated(all)
    print(no_tickets)
    for i in no_tickets:
        pass

TypeError: 'CollectionReference' object is not iterable